In [1]:
import tensorflow as tf
import numpy as np

In [2]:
X = []
t = 1
for i in range(t):
    with open("single_matrix/view" + str(i) + ".npy", 'rb') as f:
        X.append(tf.cast(tf.sparse.from_dense(np.load(f)), dtype=tf.float32))

In [3]:
k = 7
steps = 1
n = X[0].shape[1]
alpha, beta, gamma = 1.0, 1000, 0.01
wv = [1.0 / t for i in range(t)]
reg = gamma / (4.0 * beta)
I = tf.eye(n, dtype=tf.float32)
S = tf.eye(n, dtype=tf.float32)

In [14]:
for step in range(steps):
    S = tf.where(S > 0, S, tf.zeros(S.shape))
    S = (S + tf.transpose(S)) / 2.0
    Z_sum = tf.zeros(S.shape)
    for i in range(t):
        Xv = tf.sparse.to_dense(X[i])
        aux = tf.linalg.matmul(Xv, Xv, transpose_a=True)
        Z = tf.linalg.matmul(
                tf.linalg.inv(aux + I * alpha + I * beta * wv[i]),
                S * beta * wv[i] + aux)
        Z = tf.where(Z[i] > 0, Z, tf.zeros(Z.shape))
        Z = (Z + tf.transpose(Z)) / 2.0
        wv[i] = 0.5 / tf.norm(Z - S)
        Z *= wv[i]
        Z_sum += Z

    L = tf.linalg.diag(tf.math.reduce_sum(S, 0)) - S
    (_, F) = tf.linalg.eigh(L)

    p = tf.norm(F[0][:] - F, axis=1, keepdims=True)
    for i in range(1, n):
         p = tf.concat([p, tf.norm(F[i][:] - F, axis=1, keepdims=True)], 1)
    p = tf.math.square(p) * reg
    tmp = S
    S = (Z_sum - p) / np.sum(wv)
    if step > 5 and tf.norm(S - tmp) / tf.norm(tmp) < 1.e-3:
        print('Num steps:', step)
        break
print(S)
with open("fusion_matrix/S.npy", 'wb') as f:
    np.save(f, S.numpy())

In [13]:
# x = tf.constant([[2., 16., 128.],
#                  [4., 32., 256.],
#                  [8., 64., 512.]], dtype=tf.float32)

tf.Tensor(0.008587566, shape=(), dtype=float32)
0.017175132
0.017175132
